In [ ]:
# 필요한 라이브러리 설치
# !pip install langchain langchain-openai pydantic

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# OpenAI API 키 설정 (실행 환경에 맞게 설정해주세요)
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# 1. 출력 데이터 구조 정의 (Pydantic 모델)
# 사용자의 발화에서 '의도', '행동', '세부사항'을 추출하기 위한 구조
class UserIntent(BaseModel):
    """사용자의 요청을 분석하여 구조화된 데이터로 정리합니다."""
    intent: str = Field(description="사용자 요청의 핵심 의도를 한 문장으로 요약합니다.")
    action: str = Field(description="사용자가 수행하고자 하는 구체적인 행동입니다. (예: 예약, 검색, 구매 등)")
    details: dict = Field(description="요청에 포함된 구체적인 정보들을 key-value 형태로 정리합니다.")

# 2. Pydantic 출력 파서 생성
# Pydantic 모델을 기반으로 LLM의 출력을 파싱할 파서를 만듭니다.
parser = PydanticOutputParser(pydantic_object=UserIntent)

# 3. 프롬프트 템플릿 생성
# LLM에게 역할을 부여하고, 사용자의 입력을 어떻게 처리할지 지시합니다.
# 파서의 get_format_instructions()를 포함하여 출력 형식을 명시합니다.
template_text = """
당신은 사용자의 요청을 분석하고 핵심 정보를 추출하는 전문가입니다.
사용자의 발언을 바탕으로 핵심 의도, 행동, 그리고 세부 정보를 JSON 형식으로 정리해주세요.

{format_instructions}

사용자 발언:
{user_query}
"""

prompt = ChatPromptTemplate.from_template(
    template=template_text,
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 4. LLM 모델 및 체인 구성
# 안정적인 결과를 위해 temperature를 0으로 설정합니다.
model = ChatOpenAI(temperature=0, model="gpt-4")

# 프롬프트, 모델, 파서를 연결하여 체인을 완성합니다.
chain = prompt | model | parser

# --- 실행 예시 ---
user_input_1 = "다음 주 금요일 오전에 서울에서 출발해서 부산으로 가는 KTX 기차표를 예매하고 싶어. 성인 2명이야."
user_input_2 = "오늘 저녁 7시에 강남역 근처에서 4명이 먹을 만한 이탈리안 레스토랑 좀 찾아줄래? 파스타가 맛있었으면 좋겠어."

# 체인 실행 및 결과 확인
try:
    parsed_result_1 = chain.invoke({"user_query": user_input_1})
    parsed_result_2 = chain.invoke({"user_query": user_input_2})

    print("--- 입력 1 분석 결과 ---")
    print(parsed_result_1.model_dump_json(indent=2))

    print("\n--- 입력 2 분석 결과 ---")
    print(parsed_result_2.model_dump_json(indent=2))

except Exception as e:
    print(f"오류가 발생했습니다: {e}")



In [ ]:
from src.core.utils.utils import process_state_data
import json
from enum import Enum

# --- 테스트를 위한 가짜(Mock) 클래스 정의 ---
# 실제 langchain_core.messages.HumanMessage를 흉내 냅니다.
class HumanMessage:
    """테스트용 HumanMessage 클래스"""
    def __init__(self, content):
        self.content = content
    
    def __repr__(self):
        # 객체를 출력할 때 보기 좋게 표시하기 위함입니다.
        return f"HumanMessage(content='{self.content}')"

# 테스트용 Enum 정의
class TaskStatus(Enum):
    """테스트용 Enum 클래스"""
    PENDING = "대기 중"
    IN_PROGRESS = "진행 중"
    COMPLETED = "완료됨"


original_data = {
        'user_query': HumanMessage(content="AI 엔지니어 채용 공고 찾아줘"),
        'request_id': 12345,
        'tasks': [
            {
                'name': HumanMessage(content="공고 검색"),
                'status': TaskStatus.IN_PROGRESS,
                'sub_tasks': [
                    HumanMessage(content="자격 요건 분석"),
                    "키워드 추출"
                ]
            },
            {
                'name': HumanMessage(content="이력서 매칭"),
                'status': TaskStatus.PENDING
            }
        ],
        'is_urgent': False,
        'metadata': None
    }

original_data = process_state_data(original_data)
original_data